***Author: Ritwika VPS, ritwika@ucmerced.edu***  
***Written: October 9, 2025***  
###### (see below for modifications log)  

This script investigates the memory problem in the context of Collective Intelligence (CI) vs. Collective Memory (CM) by comparing how the coverage of a task consisting of TaskSize number of memory bits compares between groups that are fixed/'stable' and have performed the task before (CM groups) and groups consisting of agents randomly pooled from groups that have previously performed the task (CI groups). That is, in the latter case, the agents were part of a group that has solved the task but have not themselves solved the task together. CM and CI groups have the same number of agents with each agent having the same memory size (i.e., number of memory bits that can be stored by the agent).

> Note that there can be further complexities added to this framework, including variable memory for agents (i.e., not all agents have the same number of memory bits), noise or error in memory storage or retrival (which can further vary across agents), and memory bits that are stored exactly (when the bit is solved by the agent) and memory bits that are stored with error (when the agent observes another agent solving the bit). Here, however, we choose to stick to the much simpler case where memory bits are stored and recalled perfectly, there is only one type of memory storage (without distinguishing between observed or solved bits), and all agents have the same memory size.  

Now, consider that the total memory of the task is given by TaskSize, the number of agents i a group is N_a, and the memory size for each agent is m_a. From the perspective of the computational algorithm, there are 2 broad cases:  
- Case I. ***TaskSize $\ge$ N<sub>a</sub>m<sub>a</sub>***: This corresponds to the case where there are no redundant memory bits. That is, no memory bit of the task is stored more than once in a group of agents. In fact, if the TaskSize is less than the total memory in a group, some bits are necessarily excluded (which, in turn, would correspond to the task being partially solved by the group, which we include in the simulations for illustrative purposes) (*LeqGroupMem*).  
- Case II. ***TaskSize $<$ N<sub>a</sub>m<sub>a</sub>***: This corresponds to the case where there *are* agent memory bits that allow for redundant storage of task memory bits. Because the total memory in a group is greater than the TaskSize, some memory bits of the task are stored more than once. We go into more detail on this more below.

The case where TaskSize $\leq$ N<sub>a</sub>m<sub>a</sub> necessarily means that the task is exhaustively covered by the group because there are at least as many bits available across the group as there are bits in the task, and we are assuming that a group of agents completing the task means that the task is fully solved (and therefore, stored in the agents' memory). While this is straightforward when TaskSize is exactly equal to N<sub>a</sub>m<sub>a</sub>, there are various ways in which this can be realised when TaskSize $<$ N<sub>a</sub>m<sub>a</sub> (and especially as N<sub>a</sub>m<sub>a</sub> $>>$ TaskSize). 

For case II, there are further two broad cases:  
- Case IIa. When there is no redundancy (i.e., when some agent memory bits are not filled):  
    - Case IIa.1. when the memory bits are assigned randomly such that all agents randomly get assigned some bits (*GreaterGroupMem_NoRedundRandom*). E.g., for a task of size 5, A1 = 1 NaN 4; A2 = 2 NaN NaN; A3 = 3 5 NaN
    - Case IIa.2: first come, first serve, such that depending on how much redundancy exists, some agents get assigned no bits (*GreaterGroupMem_NoRedund1Come1Serve*). E.g., for a task of size 5, A1 = 1  2 4; A2 = 5 3 NaN; A3 = NaN NaN NaN
    - Case IIa.3: when the memory bits are assigned sequentially (to agents) and randomly such that depending on how many bits of redundancy exists, agents have roughly equal number of memory bits as NaN (*GreaterGroupMem_NoRedundRandomSeq*). E.g., for a task of size 5, A1 = 1 4 NaN; A2 = 2 3 NaN; A3 = 5 NaN NaN
- Case IIb. When there *is* redundancy, such that once all bits for the task are assigned to agents, leftover agent memory bits are assigned more bits of the task, such that task bits are not uniquely assigned to agents. If the same task memory bit is not allowed to be stored by an agent more than once, 'redundant' versions of cases IIa.1--3 collapse into a single redundant case, with any agent bits $>$ TaskSize remain unfilled. 

For case IIb, if the redundant agent memory bits (i.e., the bits that remain unfilled after exhaustively covering the TaskSize, as outlined in Case IIa) are allowed to be filled such that an agent may store more than one copy of the same task memory bit, then cases IIa.1--3 are at least somehwat distinct from each other.  

Another potential layer of complexity can be introduced by each agent drawing task memory bits independently such that unless m_a $geq$ TaskSize, there is no guarantee of the task being exhaustively covered. Within this scenario, there can further be cases where unfilled memory bits after this initial drawing (for m_a $>$ TaskSize) are filled by another draw from the task memory bits or not. 

Note that storing the same bit multiple times vs. leaving agent memory bits unfilled are equivalent within the context of what we are measuring, since we are only interested in how stable vs. mixed groups are able to represent the task, and a given task bit being stored once or multiple times within an agent's memory here does not change the outcome. 

We will treat Case IIa.1 as the primary realisable situation and treat the other possibilities as alternatives.  

>> From the perspective of how the outcomes changes, the above cases can be summarised as follows (a total of 9 variants):  
- No redundancy (unfilled agent memory bits remain unfilled) + task covered as exhaustively as possible (contingent upon how N_a, m_a, and TaskSize are related): 3 variants (Case I + Cases IIa.1--3). 
- With redundancy (unfilled agent memory bits are filled with more task bits) + exhaustive + no repeated bits at the agent level: 1 variant (Case I + Case IIb). 
- With redundancy (unfilled agent memory bits are filled with more task bits) + exhaustive + repeated bits allowed at the agent level: 3 variants (Case I + Case IIa.1--3).  
- Drawing task memory bits with replacement (i.e., potentially non-exhaustive upto m_a $<$ TaskSize): 2 variants (repeated bits allowed OR not at the agent level)

In [132]:
#Import necessary modules
import numpy as np

In [ ]:
def DistributeTask_LeqGroupMem(TaskSize, N_a, m_a):
    """
    This function distributes the task (consisting of TaskSize number of bits) among N_a agents (consisting of a group), each with memory size of m_a bits such that N_a*m_a <= TaskSize.
    That is, the total number of memory bits pooled across agents in a group can at most only fully cover the TaskSize such that there are no redundant bits of memory.

    Parameters:
    TaskSize (int): memory size of task.
    N_a (int): Number of agents in a group.
    m_a (int): Memory capacity of each agent.

    Returns:
    AgentBits (2d numpy array): each row corresponds to an agent and the elements in the row correspond to the memory bits assigned to that agent.
    NumExcludedBits (int): Number of bits of the task not covered by the agents in the group.
    """
    TotalCoverage = N_a*m_a # Total memory coverage across agents
    if TotalCoverage > TaskSize: #error check to make sure that we are within the limits of the No Redundancy condition
        raise ValueError("Total memory across agents exceeds TaskSize such that there will be redundant memory bits. Please ensure N_a * m_a <= TaskSize.")
    
    MemoryPerm = np.random.permutation(range(TaskSize)) + 1 # Randomly permute the memory vector (obtained as range(TaskSize)) (+ 1 because Python indexes from 0)
    
    if TotalCoverage == TaskSize: #if total memory across agents exactly covers the Task
        AgentBits = np.reshape(MemoryPerm, (N_a, m_a)) #Reshape the permuted memory vector into an array of N_a rows and m_a columns, such that each row corresponds to 
        #the memory bits assigned to an agent
        NumExcludedBits = np.size(MemoryPerm) - np.size(AgentBits) #Number of bits excluded 

        #error checks
        if NumExcludedBits != 0: 
            raise ValueError("Number of bits excluded should be zero when TotalCoverage is exactly equal to TaskSize.")
        
        if np.size(np.setdiff1d(MemoryPerm, AgentBits)) != 0: 
            raise ValueError("The setdiff of the AgentBits array and the MemoryPerm array/list should be empty (because the task is fully covered by pooled agent memory in the group).")
        
    elif TotalCoverage < TaskSize: #if total memory across agents is less than TaskSize
        MemoryPermTrimmed = MemoryPerm[0:TotalCoverage] #Trim the permuted memory vector to only include the first TotalCoverage number of bits
        
        AgentBits = np.reshape(MemoryPermTrimmed, (N_a, m_a)) #Reshape the trimmed permuted memory vector into an appropriately shaped array (see above)
        NumExcludedBits = np.size(MemoryPerm) - np.size(AgentBits) #Number of bits excluded (should be TaskSize - TotalCoverage in this case)
        
         #error checks
        if NumExcludedBits != (TaskSize - TotalCoverage):
            raise ValueError("Number of bits excluded should be equal to TaskSize - TotalCoverage when TotalCoverage is less than TaskSize.")
        elif NumExcludedBits < 0:
            raise ValueError("Number of bits excluded should not be negative.")
    
    return AgentBits, NumExcludedBits

In [ ]:
def DistributeTask_SuffGroupMem_NoRedundRandom(TaskSize, N_a, m_a):
    """ 
    This function distributes the task (consisting of TaskSize number of bits) among N_a agents in a group, each with memory size of m_a bits such that N_a*m_a > TaskSize. That is, the
    total number of memory bits pooled across agents in a group exceeds the TaskSize such that there are redundant bits of memory. 

    Task memory distribution is done such that bits are assigned randomly to agents. This is accomplished by padding the range(TaskSize) vector with NaNs such that the total length 
    is now equal to N_a*m_a (the total number of memory bits available in the group), randomly permuting this padded vector, and reshaping it into an array of shape (N_a, m_a) such 
    that each row corresponds to an agent and the elements in the row correspond to the memory bits assigned to that agent. By doing this, it is ensured that all agents get assigned 
    some bits (strictly speaking this would depend on how much redundancy exists, such that when N_a*m_a >> TaskSize, some agents may still get assigned no bits). Since there is no 
    redundancy, unfilled agent memory bits are assigned NaN.
    """
    
    TotalCoverage = N_a*m_a # Total memory coverage across agents
    if TotalCoverage <= TaskSize: #error check to make sure that we are within the limits of the Redundancy condition
        raise ValueError("Total memory across agents is less than TaskSize such that there will cannot be redundant memory bits. Please ensure N_a * m_a > TaskSize.")
    
    PaddedMemoryPerm = np.random.permutation(range(TotalCoverage)) + 1# Randomly permute the padded memory vector (obtained as range(TotalCoverage)) (+ 1 because Python indexes from 0)
    PaddedMemoryPerm = PaddedMemoryPerm.astype(np.float64) # Convert to float to allow NaN assignment (see below)
    PaddedMemoryPerm[PaddedMemoryPerm > TaskSize] = np.nan #Assign NaN to all memory bits that exceed TaskSize 
    AgentBits = np.reshape(PaddedMemoryPerm, (N_a, m_a)) #Reshape the permuted memory vector into an array of N_a rows and m_a columns, such that each row corresponds to 
    #the memory bits assigned to an agent

    #error checks
    TrueMemoryVec = np.arange(TaskSize) + 1 #The true memory vector (i.e., the range(TaskSize) vector) (+ 1 because Python indexes from 0)
    #print(TrueMemoryVec) #print(np.setdiff1d(TrueMemoryVec, AgentBits))
    if np.size(np.setdiff1d(TrueMemoryVec, AgentBits)) != 0: #note that setdiff1d automatically removes NaNs
        raise ValueError("The setdiff of the AgentBits array and TrueMemoryVec should be empty (because the task is fully covered by pooled agent memory in the group).")
    
    NumExcludedBits = 0 #Number of bits excluded (should be zero in this case) (and we know this because we have done error checks)
    
    return AgentBits, NumExcludedBits

In [ ]:
def DistributeTask_SuffGroupMem_NoRedund1Come1Serve(TaskSize, N_a, m_a):
    """ 
    This function distributes the task (consisting of TaskSize number of bits) among N_a agents in a group, each with memory size of m_a bits such that N_a*m_a > TaskSize. That is, the
    total number of memory bits pooled across agents in a group exceeds the TaskSize such that there are redundant bits of memory. 

    Task memory distribution is done such that bits are assigned randomly to agents on a first come first serve basis. That is, the first agent receives a randomised assortment of task 
    bits, and if there are any leftover bits, those are randomly assigned to the second agent and so on and so forth. This is accomplished by randomly permuting the range(TaskSize) 
    vector and *then* padding it with NaNs such that the total length is now equal to N_a*m_a (the total number of memory bits available in the group). This is followed by sequentially 
    assigning bits from this permuted vector to each agent by reshaping the permuted vector into an array of shape (N_a, m_a) such that each row corresponds to an agent and the elements
    in the row correspond to the memory bits assigned to that agent. By doing this, the memory distribution is such that the first one or more agents are biased to store more (or all)
    bits of the task. Since there is no redundancy, unfilled agent memory bits are assigned NaN. 
    """
    
    TotalCoverage = N_a*m_a # Total memory coverage across agents
    if TotalCoverage <= TaskSize: #error check to make sure that we are within the limits of the Redundancy condition
        raise ValueError("Total memory across agents is less than TaskSize such that there will cannot be redundant memory bits. Please ensure N_a * m_a > TaskSize.")
    
    MemoryPerm = np.random.permutation(range(TaskSize)) + 1 # Randomly permute the memory vector (obtained as range(TaskSize)) (+ 1 because Python indexes from 0)
    ArrayToPad = np.full((TotalCoverage-TaskSize,),np.nan) #Create a 1d array of NaNs to pad the permuted memory vector with. Here, the argument (TotalCoverage-TaskSize,) 
    #creates a 1D array of length (TotalCoverage-TaskSize). That is, there will be (TotalCoverage-TaskSize) number of NaNs in this array.
    PaddedMemoryPerm = np.concatenate((MemoryPerm, ArrayToPad)) #Pad the permuted memory vector with NaNs such that its length is equal to TotalCoverage
    
    #error checks
    if len(PaddedMemoryPerm) != TotalCoverage: 
        raise ValueError("The length of the padded memory vector should be equal to TotalCoverage.")
    
    if len(MemoryPerm) != TaskSize:
        raise ValueError("The length of the permuted memory vector should be equal to TaskSize.")
    
    PaddedMemoryPerm = np.random.permutation(range(TotalCoverage)) + 1# Randomly permute the padded memory vector (obtained as range(TotalCoverage)) (+ 1 because Python indexes from 0)
    PaddedMemoryPerm = PaddedMemoryPerm.astype(np.float64) # Convert to float to allow NaN assignment (see below)
    PaddedMemoryPerm[PaddedMemoryPerm > TaskSize] = np.nan #Assign NaN to all memory bits that exceed TaskSize 
    AgentBits = np.reshape(PaddedMemoryPerm, (N_a, m_a)) #Reshape the permuted memory vector into an array of N_a rows and m_a columns, such that each row corresponds to 
    #the memory bits assigned to an agent

    #error checks
    TrueMemoryVec = np.arange(TaskSize) + 1 #The true memory vector (i.e., the range(TaskSize) vector) (+ 1 because Python indexes from 0)
    #print(TrueMemoryVec) #print(np.setdiff1d(TrueMemoryVec, AgentBits))
    if np.size(np.setdiff1d(TrueMemoryVec, AgentBits)) != 0: #note that setdiff1d automatically removes NaNs
        raise ValueError("The setdiff of the AgentBits array and TrueMemoryVec should be empty (because the task is fully covered by pooled agent memory in the group).")
    
    NumExcludedBits = 0 #Number of bits excluded (should be zero in this case) (and we know this because we have done error checks)
    
    return AgentBits, NumExcludedBits

In [ ]:
MemoryPerm = np.random.permutation(range(5)) + 1
print(MemoryPerm )
np.ndim(MemoryPerm)

[5 3 1 2 4]


1

In [ ]:
AgentBits_Less, NumExcludedBits_Less = DistributeTask_NoRedund(15, 3, 5)
print(AgentBits_Less)
print(NumExcludedBits_Less)

print('--------')

AgentBits_Red, NumExcludedBits_Red = DistributeTask_RedundRandom(12, 3, 5)
print(AgentBits_Red)
print(NumExcludedBits_Red)
print(np.arange(1,13))


""" 
Once all memory bits of the task are assigned, the remaining memory bits for each agent are assigned additional task bits randomly, but with the condition that no agent 
can store the same bit more than once. Any unfilled agent memory bits are assigned NaN.
"""


[[ 9 13 11  7  2]
 [14 10  4 12  3]
 [ 8 15  5  6  1]]
0
--------
[[ 3. nan 12. nan  5.]
 [ 1.  7.  2.  8.  9.]
 [11. 10.  4.  6. nan]]
0
[ 1  2  3  4  5  6  7  8  9 10 11 12]
